In [49]:
#CSC620
#HA#6.2 -- Using NB Classification File
#Paula Abigail Tam <921850992>

import pandas as pd
from nltk import word_tokenize
import math #to use .prod

model_check = pd.read_csv('./HA6_csv/model.csv') #using pandas to read the .csv file
model_check.rename(columns={'0':'Cond. Prob.'}, inplace=True) #to rename the column with all the probabilities

df_test_set = pd.read_csv('./HA6_csv/test.csv') #read test set

df_pred = pd.DataFrame(columns=['Document', 'Predicted Class', 'Actual Class']) #prep new df for the predictions

In [50]:
def normalize_words(words): #for test set
    new_tokens = [word.casefold() for word in word_tokenize(words) if word.isalnum()] #only alphanumeric words
    return new_tokens

def multiply(list_prob): #to multiply the probabilities
    if not list_prob: #if empty list
        result = 0
    else:
        result = math.prod(list_prob) #multiply all probabilities together
    return result

for row in df_test_set.index: #row in test set
    #print(df_test_set['Document'][row], df_test_set['Class'][row])
    curr_sent = df_test_set['Document'][row] #each sentence in test
    actual_class = df_test_set['Class'][row] #the actual class of the doc
    pos_prob = [] #both inside for loops so it empties automatically
    neg_prob = []
    for test_word in normalize_words(curr_sent): #normalize and each word
        #print(test_word, " in ", curr_sent)
        for model_row in model_check.index: #each word in model
            curr_model_word = model_check['Document'][model_row]
            curr_model_class = model_check['Class'][model_row]
            curr_model_prior = model_check['Priors'][model_row]
            curr_model_prob = model_check['Cond. Prob.'][model_row]
            if test_word == curr_model_word: #if strings are the same
                #print(test_word, curr_model_word)
                if curr_model_class == model_check.Class.unique()[0]: #if the model word is positive class
                    #print(curr_model_class, curr_model_prob)
                    pos_prob.append(curr_model_prob)
                else: #if negative
                    #print(curr_model_class, curr_model_prob)
                    neg_prob.append(curr_model_prob)
            else: #if word is NOT in the model, skip it
                continue;
        
    #print("end of first sent")
    #print(curr_sent)
    #print(pos_prob)
    #print(neg_prob)
    if multiply(pos_prob)*curr_model_prior > multiply(neg_prob)*curr_model_prior: #if pos > neg
        #print("positive")
        row = {'Document': curr_sent, 'Predicted Class': "positive", 'Actual Class': actual_class}
        new_df = pd.DataFrame([row])
        df_pred = pd.concat([df_pred, new_df], ignore_index=True) #cant use pd.append bc it's being deprecated
    elif multiply(pos_prob)*curr_model_prior < multiply(neg_prob)*curr_model_prior: #if neg < pos
        #print("negative")
        row = {'Document': curr_sent, 'Predicted Class': "negative", 'Actual Class': actual_class}
        new_df = pd.DataFrame([row])
        df_pred = pd.concat([df_pred, new_df], ignore_index=True)
    else: #if equal or smth
        print("Shouldn't happen.") #corner case
    #neg, neg, pos, pos, pos, neg, neg, neg, pos, neg  #output
    #pos, neg, pos, neg, pos, pos, neg, neg, neg, pos #actual
    #print("pos_prob: ", multiply(pos_prob))
    #print("neg_prob: ", multiply(neg_prob))

In [51]:
#print(df_pred)
df_pred.to_csv('./HA6_csv/test_predictions.csv', index=False)